In [ ]:
import sys
print(sys.version)

Simulates Grocery Store Transactions and Creates Sales Table

In [ ]:
import sqlite3 as lite
import csv
import random
from sqlite3.dbapi2 import Date
import datetime

def readItems(cur, skus, log):
    cur.execute('drop table if exists items')
    cur.execute('create table items(sku INT, name TEXT, manufacturer TEXT, size TEXT, type TEXT, basePrice DECIMAL)')
    with open(r'Products1.txt') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        for i,row in enumerate(csv_reader):
            if(i == 0):
                continue
            if(log):
                print(row)
            cur.execute("INSERT INTO items (sku, name, manufacturer, size, type, basePrice) VALUES (?, ?, ?, ?, ?, ?)",
                (row[4], row[0], row[1], row[2], row[3], row[5]))
            skus.append(row[4])
        con.commit()
    if(log):
        cur.execute("SELECT * from items where sku = ?", (42081001,))
        print(cur.fetchall())

def getTypeSKU(cur, type):
    cur.execute("SELECT sku, basePrice from items where type = ?", (type,))
    rows = cur.fetchall()
    return random.choice(rows)

def buyItem(cur, count, date, row):
    priceMulti = 1.2
    count += 1
    cur.execute('INSERT INTO sales (customerNum, sku, saleDate, price) VALUES (?, ?, ?, ?)', 
        (count, row[0], date.date().strftime('%Y-%m-%d'), (round(float(row[1].strip("$")) * priceMulti, 2))))

def buyRandomItem(cur, count, date, skus):
    sku = random.choice(skus)
    cur.execute("SELECT sku, basePrice from items where sku = ?", (sku,))
    rows = cur.fetchall()
    buyItem(cur, count, date, rows[0])

def randomCustomer(dailyCount, customerNumber, date, skus):
    maxItems = 80
    itemCount = 0
    milkChance = 70
    babyFoodChance = 20
    breadChance = 50
    pbChancce = 10
    cerealChance = 5
    diaperChance = 1
    jellyChance = 5

    customerTotal = random.randrange(1, maxItems)
    #check if customer buys special items
    if(random.randrange(1, 100) < milkChance):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Milk'))
        itemCount += 1
        cerealChance = 50
    if(itemCount >= customerTotal): return customerTotal
    if(random.randrange(1, 100) < cerealChance):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Cereal'))
        itemCount += 1
    if(itemCount >= customerTotal): return customerTotal
    if(random.randrange(1,100) < babyFoodChance):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Baby Food'))
        itemCount += 1
        diaperChance = 80
    if(itemCount >= customerTotal): return customerTotal
    if(random.randrange(1, 100) < diaperChance):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Diapers'))
        itemCount += 1
    if(itemCount >= customerTotal): return customerTotal
    if(random.randrange(1, 100) < breadChance):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Bread'))
        itemCount += 1
    if(itemCount >= customerTotal): return customerTotal
    if(random.randrange(1, 100) < pbChancce):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Peanut Butter'))
        itemCount += 1
        jellyChance = 90
    if(itemCount >= customerTotal): return customerTotal
    if(random.randrange(1, 100) < jellyChance):
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Jelly/Jam'))
        itemCount += 1
    if(itemCount >= customerTotal): return customerTotal
    #buy the rest of the items randomly
    for i in range(itemCount, customerTotal):
        buyRandomItem(cur, customerNumber, date, skus)
    return customerTotal

def randomCount(WKDmin, WKDmax, WkEnd, start_date):
    #generate random customers
    customerNum = random.randint(WKDmin, WKDmax)
    #if weekend add the additional customers    
    if start_date.weekday() > 4:
        customerNum += WkEnd
    return customerNum

con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

cur.execute('drop table if exists sales')
cur.execute('create table sales(saleDate DATE, customerNum INT, sku INT, price DECIMAL)')

dailyCount = 0
skus = []
readItems(cur, skus, False)

#Start of this loop through the year
WKDmin = 1000
WKDmax = 1040
WkEndCount = 75
MaxItems = 80
PriceMultiplier = 1.05
current_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
delta = datetime.timedelta(days=1)
customerTotal = 0

while current_date <= end_date:
    dailyCount = 0
    #Get customer count for the day
    customerCount = randomCount(WKDmin, WKDmax, WkEndCount, current_date)
    #Loop through this count
    for c in range(customerCount):
        dailyCount += randomCustomer(dailyCount, c, current_date, skus)
    con.commit()
    current_date += delta
    customerTotal += customerCount

Individual Store Sales Data Report Retrieval (Store Sales.xlsx)

1. Gets Average Customers and Sales Per Day

In [ ]:
#count customers
cur.execute('SELECT count(*) FROM ( select DISTINCT saleDate, customerNum from sales)')
row = cur.fetchone()
avg_customers = row[0]/14
print('Actual Avg Customers per day: ', round(avg_customers))
#total sales
cur.execute('SELECT count(*) FROM sales')
row = cur.fetchone()
total_avg_sales = row[0]/14
print('Actual Avg Sales per day: ', round(total_avg_sales))

2. Finds the Min and Max Non-special Products 

In [ ]:
# Min and max non-special sku products
item_types = ('Milk', 'Cereal', 'Baby Food', 'Diapers', 'Peanut Butter', 'Bread', 'Jelly/Jam')
cur.execute('SELECT COUNT(*) as total_sold FROM sales JOIN items on sales.sku = items.sku WHERE items.type NOT IN (?, ?, ?, ?, ?, ?, ?) GROUP BY sales.sku ORDER BY total_sold', item_types)
rows = cur.fetchall()
print('Min # non-special sku products: ', rows[0][0])
print('Max # non-special sku products: ', rows[-1][0])

3. Finds the Sale Info of Special Products

In [ ]:
# Average Sales Per Day of Special Products
for item_type in item_types:
    cur.execute('SELECT count(sales.sku) FROM sales JOIN items ON sales.sku = items.sku WHERE items.type = ?', (item_type,))
    row = cur.fetchone()
    print('Total Sales for ', item_type, ': ', row[0])
    avg_sales = round(row[0]/14)
    print('Avg Sales Per Day for ', item_type, ': ', avg_sales)
    print('% Total: ', round((avg_sales/total_avg_sales)*100,1))
    cur.execute('SELECT COUNT(*) FROM items WHERE type = ?', (item_type,))
    row = cur.fetchone()
    print('# Items in Type: ', row[0])
    print('Sales Per Item: ', round(avg_sales/row[0],1))
    print()

4. Calculates the Sale Info of Other Products

In [ ]:
# Other Average Sales Per Day
cur.execute('SELECT COUNT(*) as total_sales FROM sales JOIN items ON sales.sku = items.sku WHERE items.type NOT IN (?, ?, ?, ?, ?, ?, ?)', item_types)
row = cur.fetchone()
print('Total Sales Other: ', row[0])
avg_other = round(row[0]/14)
print('Avg Sales Per Day Other: ', avg_other)
cur.execute('SELECT COUNT(*) as total_items FROM items WHERE items.type NOT IN (?, ?, ?, ?, ?, ?, ?)', item_types)
row = cur.fetchone()
print('# Items in Other: ', row[0])
print('Sales Per Item Other: ', round(avg_other/row[0],1))

Creates the Inventory Table and Simulates a Year

In [ ]:
import sqlite3 as lite
import datetime
import math
from sqlite3.dbapi2 import Date

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

cur.execute('DROP TABLE IF EXISTS inventory')
cur.execute('CREATE TABLE inventory(productName TEXT, itemType TEXT, SKU INT, threshold INT, numberItems INT, numberCases INT)')

# Inserting inventory
cur.execute("""
                SELECT sku, COUNT(*) as total_sales, COUNT(DISTINCT saleDate) as unique_dates
                FROM SALES
                GROUP BY sku""")

sku_data = cur.fetchall()

for row in sku_data:
    sku = row[0]
    avg_sales = row[1] / 365

    cur.execute("SELECT name, type FROM items WHERE sku = ?", (sku,))
    item_data = cur.fetchone()
    productName = item_data[0]
    itemType = item_data[1]    

    if itemType == 'Milk':
        threshold = math.floor(avg_sales * 1.5)  # For milk, start with 1.5 times average sales
    else:
        threshold = math.floor(avg_sales * 3)    # For other items, start with 3 times average sales
    numberCases = threshold // 12
         
    cur.execute('INSERT INTO inventory (productName, itemType, SKU, threshold, numberItems, numberCases) VALUES (?, ?, ?, ?, ?, ?)', 
        (productName, itemType, sku, threshold, threshold, numberCases))

# Creating transaction details table
cur.execute('DROP TABLE IF EXISTS transaction_details')
cur.execute('CREATE TABLE transaction_details(date DATE, customer_number INT, SKU INT, price DECIMAL, numberItems INT, numberCases INT)')

# Start of loop through the year
current_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2020, 12, 31)
delta = datetime.timedelta(days=1)

customer_number=0
while current_date <= end_date:
    # Check for sales on this date
    cur.execute("SELECT SKU, price, COUNT(*) as number_sold FROM sales WHERE saleDate = ? GROUP BY SKU, price", (current_date.strftime('%Y-%m-%d'),))
    sales_data = cur.fetchall()

    # Process each sale and insert into transaction_details
    for sku, price, number_sold in sales_data:
        # Get current inventory details before the sale
        cur.execute("SELECT numberItems, numberCases, itemType FROM inventory WHERE SKU = ?", (sku,))
        inventory_data = cur.fetchone()
        
        if inventory_data:
            number_items = inventory_data[0]  # Current inventory items
            number_cases = inventory_data[1]   # Current inventory cases
            item_type = inventory_data[2]      # Current item type

            # Calculate new number of items after the sale
            new_number_items = number_items - number_sold
            
            if new_number_items < 0:  # Check if the product is out of stock
                # Look for alternative products of the same type
                cur.execute("SELECT SKU, numberItems FROM inventory WHERE itemType = ? AND numberItems > 0", (item_type,))
                alternative_products = cur.fetchall()

                if alternative_products:  # If there are alternative products available
                    # Select the first alternative product (you could implement more logic to choose based on your criteria)
                    alt_sku, alt_number_items = alternative_products[0]  # Choose the first alternative
                    price = price  # You may want to adjust price logic based on your needs

                    # Update the alternative product inventory
                    new_alt_number_items = alt_number_items - number_sold

                    cur.execute('''
                        UPDATE inventory 
                        SET numberItems = ?
                        WHERE SKU = ?
                    ''', (new_alt_number_items, alt_sku))

                    # Insert the sale into transaction_details for the alternative product
                    cur.execute('''
                        INSERT INTO transaction_details (date, customer_number, SKU, price, numberItems, numberCases) 
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', (current_date.strftime('%Y%m%d'), customer_number, alt_sku, price, new_alt_number_items, inventory_data[1]))  # Use new inventory values for alt product
            else:
                # Update inventory for the original product
                cur.execute('''
                    UPDATE inventory 
                    SET numberItems = ?
                    WHERE SKU = ?
                ''', (new_number_items, sku))

                # Insert the sale into transaction_details with updated numberItems
                cur.execute('''
                    INSERT INTO transaction_details (date, customer_number, SKU, price, numberItems, numberCases) 
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (current_date.strftime('%Y%m%d'), customer_number, sku, price, new_number_items, number_cases))
                customer_number += 1

    # Inventory Check for Milk products
    cur.execute("SELECT SKU, numberItems, threshold, numberCases FROM inventory WHERE itemType = 'Milk'")
    milk_inventory_data = cur.fetchall()
    
    for sku, number_items, threshold, number_cases in milk_inventory_data:
        if number_items < threshold:
            missing = threshold - number_items
            cases_needed = (missing + 11) // 12  

            new_number_items = number_items + (cases_needed * 12)
            new_number_cases = number_cases + cases_needed  

            cur.execute('''
                UPDATE inventory 
                SET numberItems = ?, numberCases = ?
                WHERE SKU = ?
            ''', (new_number_items, new_number_cases, sku))

    # Inventory Check for non-milk products on Tuesday, Thursday, or Friday
    if current_date.weekday() in [1, 3, 4]:  # 1 = Tuesday, 3 = Thursday, 4 = Friday
        cur.execute("SELECT SKU, numberItems, threshold, numberCases FROM inventory WHERE itemType != 'Milk'")
        inventory_data = cur.fetchall()
            
        for sku, number_items, threshold, number_cases in inventory_data:
            if number_items < threshold:
                missing = threshold - number_items
                cases_needed = (missing + 11) // 12  

                new_number_items = number_items + (cases_needed * 12)
                new_number_cases = number_cases + cases_needed  

                cur.execute('''
                    UPDATE inventory 
                    SET numberItems = ?, numberCases = ?
                    WHERE SKU = ?
                ''', (new_number_items, new_number_cases, sku))

    # Commit after processing each day
    con.commit()   
    current_date += delta

# Clean up
cur.close()
con.close()

Creates a Combined Report Highlighting the Top 10 Selling Products of Each Store and the Combination (Combined Market Sales.pdf)

In [ ]:
import sqlite3 as lite
import pandas as pd

def getSale(path):
    # Connect to the database
    con = lite.connect(path)
    cur = con.cursor()
    
    cur.execute('''
        SELECT sales.sku, items.name as productName, count(sales.sku) as NUM 
        FROM sales 
        JOIN items ON sales.sku = items.sku
        WHERE sales.saleDate BETWEEN '2023-01-01' AND '2023-01-31'
        GROUP BY sales.sku
        ORDER BY NUM DESC
    ''')
    
    sales = cur.fetchall()
    
    # Clean up
    cur.close()
    con.close()
    
    return sales

def addSales(db1, db2): # db1 = tim; db2 = jacob

    salesDB1 = getSale(db1) # tim
    salesDB2 = getSale(db2) # jacob

    # Convert sales data to DataFrames
    db1 = pd.DataFrame(salesDB1, columns=['sku', 'productName', 'NUM'])
    db2 = pd.DataFrame(salesDB2, columns=['sku', 'productName', 'NUM'])

    # Combine the data for aggregated totals
    combined_data = pd.concat([db1, db2])

    aggregated_data = combined_data.groupby(['sku', 'productName'], as_index=False).agg({'NUM': 'sum'})
    
    aggregated_data = aggregated_data.sort_values(by='NUM', ascending=False)

    totalsales_db1 = db1['NUM'].sum()
    totalsales_db2 = db2['NUM'].sum()
    totalsales_combined = aggregated_data['NUM'].sum()

    return aggregated_data, db1, db2, totalsales_db1, totalsales_db2, totalsales_combined



db1 = 'tims_store.db'       
db2 = 'jacobs_store.db'  

totSales, timDB, jacobDB, totalSalesDB1, totalSalesDB2, totalSalesCombined = addSales(db1, db2)

combined = totSales.head(10)

timOrder = combined.merge(timDB, on=['sku', 'productName'], how='left', suffixes=('', '_tim')).fillna(0)
jacobOrder = combined.merge(jacobDB, on=['sku', 'productName'], how='left', suffixes=('', '_jacob')).fillna(0)


print(f"Total Sales for Tim's Store: {totalSalesDB1}")
print(f"Total Sales for Jacob's Store: {totalSalesDB2}")


print("Combined Top 10")
print("sku | productName | #sold | rank")
rank = 1
for _, sale in combined.iterrows():
    print(f"{sale['sku']} | {sale['productName']} | {sale['NUM']} | {rank}")
    rank += 1

# Tim
print("Tim")
print("sku | productName | #sold | rank")
rank = 1
for _, sale in timOrder.iterrows():
    print(f"{sale['sku']} | {sale['productName']} | {int(sale['NUM_tim'])} | {rank}")
    rank += 1
    
# Jacob
print("Jacob")
print("sku | productName | #sold | rank")
rank = 1
for _, sale in jacobOrder.iterrows():
    print(f"{sale['sku']} | {sale['productName']} | {int(sale['NUM_jacob'])} | {rank}")
    rank += 1

Creates New Products Table and Creates Source Table

In [ ]:
import csv
import sqlite3

def insertItems(db):
    con = sqlite3.connect(db)
    cur = con.cursor()
    
    cur.execute('drop table if exists products')
    cur.execute('''create table products(
        productKey INT,
        sku INT, 
        product_Name TEXT, 
        product_Class_ID int,
        subcategory TEXT,
        category TEXT,
        department TEXT,
        size TEXT, 
        number_per_case INT,
        brand_name TEXT,
        man TEXT, 
        supplier TEXT
    )''')

    cur.execute('drop table if exists source')
    cur.execute('''create table source(
        sku INT,
        source INT
    )''')
    
    with open(r'Products1.txt') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')

        productKey = -2
        for row in (csv_reader):
            if productKey == -2:
                productKey += 1
                continue
                
            sku = row[4]
            size = row[2]
            name = row[1]
            productName = name
            itemType = row[3]
            man = row[0]
            department = 'None'
            source = 0

            # Map Categories
            
            if itemType: # has itemType
                category_subcategory = itemType.split('/', 1)

                category = category_subcategory[0]
                subcategory = category_subcategory[1] if len(category_subcategory) > 1 else 'None'

                if subcategory == 'None':
                    # Categorizing
    
                    source = 3
                    match itemType:
                        case 'Yogurt':
                            subcategory = 'Yogurt'
                            category = 'Dairy'
                        case 'Salad Dressing':
                            subcategory = 'Salad Dressing'
                            category = 'Condiments'
                        case 'Pet Food':
                            if 'Treats' in productName: 
                                subcategory = 'Treats'
                                category = 'Pet Food'
                            else:
                                subcategory = 'Food'
                                category = 'Pet Food'
                        case 'Cereal':
                            subcategory = 'Cereal'
                            category = 'Breakfast'
                        case 'Mac & Cheese':
                            subcategory = 'Mac & Cheese'
                            category = 'Packaged Foods'
                        case 'Juice':
                            subcategory = 'Juice'
                            category = 'Drinks'
                        case 'Nuts':
                            subcategory = 'Nuts'
                            category = 'Snacks'
                        case 'Fresh Vegetables':
                            subcategory = 'Fresh Vegetables'
                            category = 'Produce'
                        case 'Fresh Fruit':
                            subcategory = 'Fresh Fruit'
                            category = 'Produce'
                        case 'Canned Fruit':
                            subcategory = 'Canned Fruit'
                            category = 'Packaged Foods'
                        case 'Diapers':
                            subcategory = 'Diapers'
                            category = 'Baby'
                        case 'Chocolate Candy':
                            subcategory = 'Chocolate'
                            category = 'Snacks'
                        case 'Hard Candy':
                            subcategory = 'Hard Candy'
                            category = 'Snacks'
                        case 'Baked Goods Other than Bread':
                            source = 2
                            if 'Bagel' in productName:
                                subcategory = 'Bagels'
                                category = 'Bakery'
                            else:
                                subcategory = 'Pastries'
                                category = 'Bakery'
                        case 'Tofu':
                            subcategory = 'Tofu'
                            category = 'Produce'
                        case 'Cooking Oil':
                            subcategory = 'Cooking Oil'
                            category = 'Oil'
                        case 'Fresh Seafood':
                            subcategory = 'Fish'
                            category = 'Seafood'
                        case 'Sugar':
                            subcategory = 'Sugar'
                            category = 'Baking'
                        case 'Peanut Butter':
                            subcategory = 'Peanut Butter'
                            category = 'Spreads'
                        case 'Spices':
                            subcategory = 'Spices'
                            category = 'Condiments'
                        case 'Cleaners':
                            if 'Glass' in productName:
                                subcategory = 'Glass'
                                category = 'Cleaner'
                            elif 'Counter' in productName:
                                subcategory = 'Counter'
                                category = 'Cleaner'
                            else:
                                subcategory = 'Toilet'
                                category = 'Cleaner'
                        case 'Fresh Fish':
                            subcategory = 'Fresh Fish'
                            category = 'Seafood'
                        case 'Deodorizers':
                            subcategory = 'Room'
                            category = 'Deodorizers'
                        case 'Bread':
                            subcategory = 'Sliced Bread'
                            category = 'Bakery'
                        case 'Frozen Food':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Acetominifen':
                            subcategory = 'Fever'
                            category = 'Medicine'
                        case 'Ibuprofen':
                            subcategory = 'Fever'
                            category = 'Medicine'
                            source = 2
                        case 'Aspirin':
                            subcategory = 'Fever'
                            category = 'Medicine'
                            source = 2
                        case 'Mouthwash':
                            subcategory = 'Mouthwash'
                            category = 'Hygiene'
                        case 'Hamburger':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Rice':
                            subcategory = 'Grain'
                            category = 'Rice'
                        case 'Milk':
                            subcategory = 'Milk'
                            category = 'Dairy'
                        case 'Soda':
                            subcategory = 'Soda'
                            category = 'Drinks'
                        case 'Snacks':
                            source = 2
                            if ('Pringles' == man or 'Frito Lay' == man):
                                subcategory = 'Chips'
                                category = 'Snacks'
                            elif ('Quaker' == man and 'Bars' in productName):
                                subcategory = 'Bars'
                                category = 'Snacks'
                            elif ('Bar' in productName):
                                subcategory = 'Bars'
                                category = 'Snacks'
                            elif ('Cookies' in productName):
                                subcategory = 'Cookies'
                                category = 'Snacks'
                            elif ('Drakes' == man or 'All But Gluten' == man):
                                subcategory = 'Cakes'
                                category = 'Snacks'
                            elif ('Chex Mix' == man):
                                subcategory = 'Nuts'
                                category = 'Snacks'
                            elif ('Fruit Snacks' in productName):
                                subcategory = 'Fruit Snacks'
                                category = 'Snacks'
                            elif ('Chex Mix' == man):
                                subcategory = 'Nuts'
                                category = 'Snacks'
                            elif ('Kroger' == man):
                                subcategory = 'Bread Sticks'
                                category = 'Snacks'
                            elif ('Kelloggs' == man or 'Red Oval Farms' == man):
                                subcategory = 'Crisps'
                                category = 'Snacks'
                            elif ('Rolled Gold' == man):
                                subcategory = 'Pretzels'
                                category = 'Snacks'
        
                        case 'Candy':
                            subcategory = 'Chocolate'
                            category = 'Snacks'
                        case 'Bologna':
                            subcategory = 'Meats'
                            category = 'Deli'
                        case 'Hot Dogs':
                            subcategory = 'Hot Dogs'
                            category = 'Meats'
                        case 'Deli Salads':
                            subcategory = 'Salads'
                            category = 'Deli'
                        case 'Deli Meats':
                            subcategory = 'Meats'
                            category = 'Deli'
                        case 'Fresh Chicken':
                            subcategory = 'Chicken'
                            category = 'Meats'
                        case 'Produce':
                            subcategory = 'Vegetables'
                            category = 'Produce'
                        case 'Soup':
                            subcategory = 'Soup'
                            category = 'Packaged Foods'
                        case 'Canned Vegetables':
                            subcategory = 'Vegetables'
                            category = 'Packaged Foods'
                        case 'Tuna':
                            subcategory = 'Fish'
                            category = 'Packaged Foods'
                        case 'TV Dinner':
                            source = 2
                            if 'Chicken' in productName:
                                subcategory = 'Chicken'
                                category = 'Packaged Foods'
                            elif 'Beef' in productName:
                                subcategory = 'Beef'
                                category = 'Packaged Foods'
                            else:
                                subcategory = 'Turkey'
                                category = 'Packaged Foods'
                        case 'French Fries':
                            subcategory = 'French Fries'
                            category = 'Packaged Foods'
                        case 'Frozen Vegetables':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Pizza':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Frozen Chicken':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Popsicles':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Ice Cream':
                            subcategory = 'Frozen Food'
                            category = 'Frozen'
                        case 'Baking Supplies':
                            subcategory = 'Milk'
                            category = 'Baking'
                        case 'Stuffing':
                            subcategory = 'Stuffing'
                            category = 'Baking'
                        case 'Dips':
                            subcategory = 'Dips'
                            category = 'Condiments'
                        case 'Popcorn':
                            subcategory = 'Popcorn'
                            category = 'Snacks'
                        case 'Cookies':
                            subcategory = 'Cookies'
                            category = 'Snacks'
                        case 'Cheese':
                            subcategory = 'Cheese'
                            category = 'Dairy'
                        case 'Cottage Cheese':
                            subcategory = 'Cheese'
                            category = 'Dairy'
                        case 'Sour Cream':
                            subcategory = 'Sour Cream'
                            category = 'Dairy'
                        case 'Baby Food':
                            subcategory = 'Food'
                            category = 'Baby'
                        case 'Cake Snacks':
                            subcategory = 'Cakes'
                            category = 'Snacks'
                        case 'Canned Goods':
                            if 'Campbells' == man:
                                subcategory = 'Soup'
                                category = 'Packaged Foods'
                            else:
                                subcategory = 'Soup'
                                category = 'Packaged Foods'
                        case 'Sardines':
                            subcategory = 'Sardines'
                            category = 'Packaged Foods'
                        case 'Drink':
                            if '4C' == man:
                                subcategory = 'Mix'
                                category = 'Drinks'
                                source = 2
                            else:
                                subcategory = 'Milk'
                                category = 'Drinks'
                        case 'Household':
                            subcategory = 'Detergent'
                            category = 'Laundry'
                else: # has both item types
                    source = 1
                    
            else: # null itemType
                source = 3
                
                if 'Yucatan' in man:
                    category = 'Dip'
                    subcategory = 'Guacomole'
                    source = 2
                elif 'burger' in name:
                    category = 'Frozen'
                    subcategory = 'Frozen Food'
                elif 'Welchs' in man:
                    category = 'Produce'
                    subcategory = 'Fruit'
                    source = 2
                elif 'Hot Cocoa Mix' in name:
                    category = 'Drinks'
                    subcategory = 'Hot Cocoa'
                elif 'Sunny' in man:
                    category = 'Drinks'
                    subcategory = 'Punch'
                elif 'Sauce' in name:
                    category = 'Dips'
                    subcategory = 'Sauce'
                elif 'Stuffing' in name:
                    category = 'Condiments'
                    subcategory = 'Stuffing'
                elif 'Powerade' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                    source = 2
                elif 'Tea' in name:
                    category = 'Drinks'
                    subcategory = 'Tea'
                elif 'Steak' in name:
                    category = 'Deli'
                    subcategory = 'Meats'
                elif 'Tuna' in name:
                    category = 'Seafood'
                    subcategory = 'Fish'
                elif 'Vinegar' in name:
                    category = 'Baking'
                    subcategory = 'Vinegar'
                elif 'Sparkling' in man:
                    category = 'Drinks'
                    subcategory = 'Water'
                elif 'Hair' in name:
                    category = 'Hygiene'
                    subcategory = 'Hair'
                elif 'Lotion' in name:
                    category = 'Hygiene'
                    subcategory = 'Skin'
                elif 'Snyders' in man:
                    category = 'Snacks'
                    subcategory = 'Chips'
                    source = 2
                elif 'Nabisco' in man:
                    if 'Chips' in name:
                        category = 'Snacks'
                        subcategory = 'Cookies'
                    else:
                        category = 'Snacks'
                        subcategory = 'Chips'
                elif 'Bagel' in name:
                    category = 'Bakery'
                    subcategory = 'Bagel'
                elif 'Goldfish' in name:
                    category = 'Snacks'
                    subcategory = 'Chips'
                    source = 2
                elif 'Cleaner' in name:
                    if 'All' in name:
                        category = 'Cleaner'
                        subcategory = 'All'
                    else:
                        category = 'Cleaner'
                        subcategory = 'Shower'
                elif 'Muffins' in name:
                    category = 'Baking'
                    subcategory = 'Muffins'
                elif 'Jiffy' in man:
                    category = 'Baking'
                    subcategory = 'Mixes'
                    source = 2
                elif 'Mix' in name:
                    category = 'Drinks'
                    subcategory = 'Mixes'
                elif 'Juice' in name:
                    category = 'Drinks'
                    subcategory = 'Juice'
                elif 'Dressing' in name:
                    category = 'Condiments'
                    subcategory = 'Salad Dressing'
                elif 'Toast' in name:
                    category = 'Bakery'
                    subcategory = 'Toast'
                elif 'Kool Aid' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                    source = 2
                elif 'Hostess' in man:
                    category = 'Snacks'
                    subcategory = 'Cakes'
                    source = 2
                elif 'Hostees' in man:
                    category = 'Snacks'
                    subcategory = 'Cakes'
                    source = 2
                elif 'Little Debbie' in man:
                    category = 'Bakery'
                    subcategory = 'Cakes'
                    source = 2
                elif 'Hershey' in man:
                    category = 'Baking'
                    subcategory = 'Syrup'
                    source = 2
                elif 'Hot Dogs' in name:
                    category = 'Frozen'
                    subcategory = 'Frozen Food'
                elif 'Sargento' in man:
                    category = 'Dairy'
                    subcategory = 'Cheese'
                    source = 2
                elif 'Oxi' in man:
                    category = 'Laundry'
                    subcategory = 'Detergent'
                    source = 2
                elif 'Popcorn' in name:
                    category = 'Snacks'
                    subcategory = 'Popcorn'
                elif 'Eggs' in name:
                    category = 'Produce'
                    subcategory = 'Eggs'
                elif 'Mac' in name:
                    category = 'Packaged Foods'
                    subcategory = 'Mac & Cheese'
                elif 'Baby' in name:
                    category = 'Hygiene'
                    subcategory = 'Lotion'
                    department = 'Baby'
                elif 'Rice' in name:
                    category = 'Rice'
                    subcategory = 'Grain'
                elif 'Capri Sun' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                    source = 2
                elif 'Clorox' in man:
                    category = 'Cleaner'
                    subcategory = 'All'
                    source = 2
                elif 'Chicken' in name:
                    category = 'Frozen'
                    subcategory = 'Frozen Food'
                elif 'Cape' in man:
                    category = 'Snacks'
                    subcategory = 'Chips'
                    source = 2
                elif 'Snapple' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                    source = 2
                elif 'Morton' in man:
                    category = 'Condiments'
                    subcategory = 'Salt'
                elif 'Seasoning' in name:
                    category = 'Condiments'
                    subcategory = 'Seasoning'
                elif 'Spread' in name:
                    category = 'Condiments'
                    subcategory = 'Spread'
                elif 'Dog' in name:
                    category = 'Pet Food'
                    subcategory = 'Dog'
                elif 'Gatorade' in man:
                    category = 'Drinks'
                    subcategory = 'Sport'
                    source = 2
                elif 'Lemonade' in name:
                    category = 'Drinks'
                    subcategory = 'Lemonade'
                elif 'Edwards' in man:
                    category = 'Bakery'
                    subcategory = 'Pie'
                    source = 2
                elif 'Glutino' in man:
                    category = 'Bakery'
                    subcategory = 'Cookies'
                    source = 2
                elif 'Minute' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                    source = 2
                elif 'Cough' in name:
                    category = 'Medicine'
                    subcategory = 'Congestion'
                elif 'Beefaroni' in man:
                    category = 'Packaged Foods'
                    subcategory = 'Macaroni'
                    source = 2
                elif 'Beef' in name:
                    category = 'Frozen'
                    subcategory = 'Frozen Foods'
                elif 'Lean' in man:
                    category = 'Frozen'
                    subcategory = 'Frozen Foods'
                elif 'Got Milk' in man:
                    category = 'Snacks'
                    subcategory = 'Flavored Straws'
                    source = 2
                elif 'Meatball' in name:
                    category = 'Frozen'
                    subcategory = 'Frozen Foods'
                elif 'Mashed' in name:
                    category = 'Dairy'
                    subcategory = 'Cheese'
                elif 'Punch' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                elif 'Crest' in man:
                    category = 'Hygiene'
                    subcategory = 'Toothpaste'
                    source = 2
                elif 'Scotch' in man:
                    category = 'Cleaner'
                    subcategory = 'Sponge'
                    source = 2
                elif 'Rowan' in man:
                    category = 'Condiments'
                    subcategory = 'Sugar'
                    source = 2
                elif 'Sour Cream' in name:
                    category = 'Dairy'
                    subcategory = 'Sour Cream'
                elif 'Flour' in name:
                    category = 'Baking'
                    subcategory = 'Flour'
                elif 'Motrin' in man:
                    category = 'Medicine'
                    subcategory = 'Fever'
                    source = 2
                elif 'Febreze' in man:
                    category = 'Deodorizer'
                    subcategory = 'Room'
                    source = 2
                elif 'Mccormick' in man:
                    category = 'Baking'
                    subcategory = 'Vanilla'
                    source = 2
                elif 'Oregon' in man:
                    category = 'Packaged Foods'
                    subcategory = 'Fruit'
                    source = 2
                elif 'Del Monte' in man:
                    category = 'Packaged Foods'
                    subcategory = 'Fruit'
                    source = 2
                elif 'Purnell' in man:
                    category = 'Frozen'
                    subcategory = 'Frozen Food'
                    source = 2
                elif 'Seed' in name:
                    category = 'Bakery'
                    subcategory = 'Sliced Bread'
                    source = 2
                elif 'Rotel' in name:
                    category = 'Produce'
                    subcategory = 'Vegetables'
                    source = 2
                elif 'Purnell' in man:
                    category = 'Frozen'
                    subcategory = 'Frozen Foods'
                    source = 2
                elif 'Palmolive' in man:
                    category = 'Cleaner'
                    subcategory = 'Dish Soap'
                    source = 2
                elif 'Ore Ida' in man:
                    category = 'Frozen'
                    subcategory = 'Frozen Food'
                    source = 2
                elif 'Ocean Spray' in man:
                    category = 'Snacks'
                    subcategory = 'Raisin'
                    source = 2
                elif 'Nestle' in man:
                    category = 'Baking'
                    subcategory = 'Milk'
                    source = 2
                elif 'Mrs. Buttersworths' in man:
                    category = 'Condiments'
                    subcategory = 'Syrup'
                    source = 2
                elif 'Motts' in man:
                    category = 'Snacks'
                    subcategory = 'Applesauce'
                    source = 2
                elif 'Little Tummys' in man:
                    category = 'Medicine'
                    subcategory = 'Gas'
                    source = 2
                elif 'Little Noses' in man:
                    category = 'Medicine'
                    subcategory = 'Nose'
                    source = 2
                elif 'Listerine' in man:
                    category = 'Hygiene'
                    subcategory = 'Condiments'
                    source = 2
                elif 'Kraft' in man:
                    category = 'Condiments'
                    subcategory = 'Whipcream'
                    source = 2
                elif 'Koch Foods' in man:
                    category = 'Frozen'
                    subcategory = 'Frozen Foods'
                    source = 2
                elif 'Jumex' in man:
                    category = 'Drinks'
                    subcategory = 'Juice'
                    source = 2
                elif 'Herdez' in man:
                    category = 'Condiments'
                    subcategory = 'Dip'
                    source = 2
                elif 'Heinz' in man:
                    category = 'Condiments'
                    subcategory = 'Dip'
                    source = 2
                elif 'Glaceau' in man:
                    category = 'Drinks'
                    subcategory = 'Vitamin Water'
                    source = 2
                elif 'Gefen' in man:
                    category = 'Condiments'
                    subcategory = 'Salt'
                    source = 2
                elif 'Dole' in man:
                    category = 'Produce'
                    subcategory = 'Fruit'
                    source = 2
            # Insert Department
            match category:
                case 'Dairy':
                    department = 'Food'
                case 'Condiments':
                    department = 'Pantry'
                case 'Breakfast':
                    department = 'Food'
                case 'Baking':
                    department = 'Pantry'
                case 'Packaged Foods':
                    department = 'Food'
                case 'Drinks':
                    department = 'Food'
                case 'Snacks':
                    department = 'Food'
                case 'Produce':
                    department = 'Food'
                case 'Bakery':
                    department = 'Food'
                case 'Spreads':
                    department = 'Food'
                case 'Oils':
                    department = 'Food'
                case 'Oil':
                    department = 'Food'
                case 'Seafood':
                    department = 'Food'
                case 'Cleaner':
                    department = 'Food'
                case 'Hygiene':
                    department = 'Health'
                case 'Frozen':
                    department = 'Food'
                case 'Deodorizer':
                    department = 'Household'
                case 'Deodorizers':
                    department = 'Household'
                case 'Rice':
                    department = 'Food'
                case 'Deli':
                    department = 'Food'
                case 'Meat':
                    department = 'Food'
                case 'Laundry':
                    department = 'Household'
                case 'Dip':
                    department = 'Food'
                case 'Dips':
                    department = 'Food'
                case 'Coffee':
                    department = 'Food'
                case 'Soda':
                    department = 'Food'
                case 'Jelly':
                    department = 'Food'
                case 'Gravy':
                    department = 'Food'
                case 'Meats':
                    department = 'Food'
                case 'Other Dairy':
                    department = 'Food'
                case 'Pasta':
                    department = 'Food'
                case 'Medicine':
                    department = 'Health'
                case 'Cake':
                    department = 'Food'
                case 'Baby':
                    if category == 'Diaper':
                        department = 'Health'
                    else:
                        department = 'Food'
                case 'Pet Food':
                    department = 'Pet'
                case _:
                    department = 'None'

            # Product Family, productKey, brand, supplier, classID
            product_family = department
            productKey += 1
            brand = man
            
            if category == 'Dairy':
                supplier = 'Rowan Dairy'
            else:
                supplier = 'Rowan Warehouse'
                
            match department:
                case 'Food':
                    classID = 0
                case 'Health':
                    classID = 1
                case 'Household':
                    classID = 2
                case 'Pantry':
                    classID = 3
                case 'Pet':
                    classID = 4
                case _: 
                    classID = -1

            # Insert into products table
            cur.execute('''INSERT INTO products (  
                    productKey,
                    sku, 
                    product_Name, 
                    product_Class_ID,
                    subcategory,
                    category,
                    department,
                    size, 
                    number_per_case,
                    brand_name,
                    man, 
                    supplier) 
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                (productKey, sku, name, classID, subcategory, category, department, size, 12, brand, man, supplier))

            # Insert into source table
            cur.execute(''' INSERT INTO source (
                    sku,
                    source)
                    VALUES (?, ?)''',
                (sku, source))
        con.commit()
        # Clean up
        cur.close()
        con.close()

insertItems('jacobs_store.db')

Ensures No Null or None Values in the Products Table

In [ ]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('jacobs_store.db')
cursor = conn.cursor()

def print_null_rows(table_name):
    # Fetch all column names from the table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [col[1] for col in cursor.fetchall()]

    # Construct a query to find rows with any NULL or 'None' values in any column
    conditions = " OR ".join([f"{column} IS NULL OR {column} = 'None'" for column in columns])
    query = f"SELECT * FROM {table_name} WHERE {conditions}"

    # Execute the query
    cursor.execute(query)
    null_rows = cursor.fetchall()

    if null_rows:
        print(f"Rows with NULL or 'None' values in '{table_name}':")
        for row in null_rows:
            print(row)
    else:
        print(f"No rows with NULL or 'None' values in '{table_name}'.")

# Usage
print_null_rows('products')

Creates the Store Dimension Table

In [ ]:
import sqlite3 as lite
import datetime
import math
from sqlite3.dbapi2 import Date

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

# Create store dimension table
cur.execute('DROP TABLE IF EXISTS store')
cur.execute('CREATE TABLE store(StoreKey INTEGER PRIMARY KEY, StoreManager TEXT, StoreStreetAddr TEXT, StoreTown TEXT, StoreZipCode TEXT, StorePhone TEXT, StoreState TEXT)')

# Only inserting our store's data as the other group did not respond to our contact
cur.execute('''
    INSERT INTO store(StoreKey, StoreManager, StoreStreetAddr, StoreTown, StoreZipCode, StorePhone, StoreState) 
    VALUES (8,'Jacob Schmitt', 'One Lincoln Financial Field Way', 'Philadelphia', '19148', '2154632500', 'PA')
''')

# Ensure data insertion worked
cur.execute('SELECT * FROM store')
rows = cur.fetchall()
for row in rows:
    print(row)


con.commit()
cur.close()
con.close()

Creates the Date Dimension Table

In [ ]:
import sqlite3 as lite
import datetime
import math
from sqlite3.dbapi2 import Date

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

# Create date dimension table
cur.execute('DROP TABLE IF EXISTS date')
cur.execute('CREATE TABLE date(DateKey INTEGER PRIMARY KEY, Date DATE, DayNumberInMonth INTEGER, DayNumberInYear INTEGER, WeekNumberInYear INTEGER, MonthNum INTEGER, MonthTxt TEXT, Quarter INTEGER, Year INTEGER, FiscalYear INTEGER, isHoliday BOOLEAN, isWeekend BOOLEAN, Season TEXT)')

# Fill the table with data about the dates
holidays = ['2023-01-01','2023-01-16','2023-02-20','2023-05-29','2023-06-16','2023-07-04','2023-09-04','2023-10-09','2023-11-11','2023-11-23','2023-12-25']
day = datetime.date(2023,1,1)
end = datetime.date(2023,12,31)
next_day = datetime.timedelta(days=1)
date_key = 1
while day <= end:
    day_number_month = day.day
    day_number_year = day.timetuple().tm_yday
    week_number = day.isocalendar()[1]
    month_num = day.month
    month_txt = day.strftime('%B')
    quarter = (month_num - 1) // 3 + 1
    year = day.year
    fiscal_year = year if month_num < 7 else year+1
    is_holiday = day.strftime('%Y-%m-%d') in holidays
    is_weekend = day.weekday() >= 5
    spring = datetime.date(year,3,20)
    summer = datetime.date(year,6,21)
    fall = datetime.date(year,9,22)
    winter = datetime.date(year,12,21)
    if spring <= day < summer:
        season = 'Spring'
    elif summer <= day < fall:
        season = 'Summer'
    elif fall <= day < winter:
        season = 'Fall'
    else:
        season = 'Winter'
    cur.execute('''
        INSERT INTO date (DateKey, Date, DayNumberInMonth, DayNumberInYear, WeekNumberInYear, MonthNum, MonthTxt, Quarter, Year, FiscalYear, isHoliday, isWeekend, Season)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (date_key, day, day_number_month, day_number_year, week_number, month_num, month_txt, quarter, year, fiscal_year, is_holiday, is_weekend, season))
    date_key += 1
    day += next_day

# Ensure data insertion worked
cur.execute('SELECT * FROM date LIMIT 20')
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
cur.close()
con.close()

Creates the Sales Fact Table at the Transaction Level

In [ ]:
import sqlite3 as lite
import datetime
import math
from sqlite3.dbapi2 import Date

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

# Create Sales Fact (Transaction Level) table
cur.execute('DROP TABLE IF EXISTS SalesFactTrans')
cur.execute('CREATE TABLE SalesFactTrans(DateKey INTEGER, DailyCustNum INTEGER, ProductKey INTEGER, StoreKey INTEGER, QuantitySold INTEGER, TotalDollarSales DECIMAL, TotalCostToStore DECIMAL, GrossProfit DECIMAL)')

# Get sales for December
cur.execute('''
    SELECT s.saleDate, p.productKey, s.price, COUNT(s.sku) AS quantity_sold, s.customerNum
    FROM sales s
    INNER JOIN products p ON s.sku = p.sku
    WHERE strftime("%m", saleDate) = "12"
    GROUP BY s.saleDate, p.ProductKey, s.price, s.customerNum
''')
sales_data = cur.fetchall()

for sale_date, product_key, sale_price, quantity_sold, customer_number in sales_data:
    # Get DateKey from date table
    cur.execute('SELECT DateKey FROM date WHERE Date = ?', (sale_date,))
    date_key_data = cur.fetchone()
    date_key = date_key_data[0]
    # Get base price from items table for the given product
    cur.execute('SELECT basePrice FROM items WHERE SKU = (SELECT sku FROM products WHERE productKey = ?)', (product_key,))
    base_price_data = cur.fetchone()
    base_price = base_price_data[0]
    # There is only one store key is the other group did not respond to our communication
    store_key = 8
    # Calculate metrics
    base_price = float(base_price.replace("$",""))
    total_dollar_sales = float(sale_price) * quantity_sold
    total_cost_to_store = base_price * quantity_sold
    gross_profit = total_dollar_sales - total_cost_to_store
    # Insert data into the SalesFact table
    cur.execute('''
        INSERT INTO SalesFactTrans (DateKey, DailyCustNum, ProductKey, StoreKey, QuantitySold, TotalDollarSales, TotalCostToStore, GrossProfit)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (date_key, customer_number, product_key, store_key, quantity_sold, total_dollar_sales, total_cost_to_store, gross_profit))

# Ensure data insertion worked
cur.execute('SELECT * FROM SalesFactTrans LIMIT 20')
rows = cur.fetchall()
for row in rows:
    print(row)

# Commit the changes and close the connection
con.commit()
cur.close()
con.close()

Creates the Sales Fact Table at the Daily Level

In [ ]:
import sqlite3 as lite
import datetime
import math
from sqlite3.dbapi2 import Date

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

# Create Sales Fact table
cur.execute('DROP TABLE IF EXISTS SalesFact')
cur.execute('CREATE TABLE SalesFact(DateKey INTEGER, ProductKey INTEGER, StoreKey INTEGER, NumSoldToday INTEGER, CostOfItemsSold DECIMAL, SalesTotal DECIMAL, GrossProfit DECIMAL)')

# Get the data
cur.execute('''
    SELECT 
        d.DateKey, 
        p.productKey, 
        st.StoreKey,
        s.saleDate,
        COUNT(s.sku) AS NumSoldToday,
        i.basePrice,
        s.price
    FROM sales s
    INNER JOIN date d ON s.saleDate = d.Date
    INNER JOIN products p ON s.sku = p.sku
    INNER JOIN items i ON s.sku = i.sku
    CROSS JOIN store st
    GROUP BY d.DateKey, p.productKey, st.StoreKey, s.saleDate, i.basePrice, s.price
''')
sales_data = cur.fetchall()

# Aggregate the data
for row in sales_data:
    date_key = row[0]
    product_key = row[1]
    store_key = row[2]
    num_sold_today = row[4]
    base_price = row[5]
    base_price = float(base_price.replace("$",""))
    sale_price = float(row[6])
    cost_of_items_sold = base_price * num_sold_today
    sales_total = sale_price * num_sold_today
    gross_profit = sales_total - cost_of_items_sold
    cur.execute('''
        INSERT INTO SalesFact (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemsSold, SalesTotal, GrossProfit)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (date_key, product_key, store_key, num_sold_today, cost_of_items_sold, sales_total, gross_profit))

# Ensure data insertion worked
cur.execute('SELECT * FROM SalesFact LIMIT 20')
rows = cur.fetchall()
for row in rows:
    print(row)

# Commit changes and close the connection
con.commit()
cur.close()
con.close()

Creates the Inventory Fact Table at the Daily Level

In [ ]:
import sqlite3 as lite
import datetime
import math
from sqlite3.dbapi2 import Date

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

# Create Inventory Fact table
cur.execute('DROP TABLE IF EXISTS InventoryFact')
cur.execute('CREATE TABLE InventoryFact(DateKey INTEGER, ProductKey INTEGER, StoreKey INTEGER, NumAvailable INTEGER, CostToStoreItemLevel DECIMAL, CostToStoreCaseLevel DECIMAL, NumCasesPurchasedToDate INTEGER)')

# There is only one store key is the other group did not respond to our communication
store_key = 8
# Get data for aggregation
cur.execute('''
    SELECT 
        d.DateKey,
        p.productKey,
        st.StoreKey,
        inv.numberItems AS NumAvailable,
        i.basePrice AS CostToStoreItemLevel,
        SUM(td.numberCases) AS NumCasesPurchasedToDate,
        inv.numberCases AS NumAvailableCases
    FROM inventory inv
    INNER JOIN products p ON inv.sku = p.sku
    INNER JOIN items i ON inv.sku = i.sku
    INNER JOIN transaction_details td ON inv.sku = td.sku
    INNER JOIN date d ON td.date <= d.Date -- Match or track up to specific date in transaction_details
    CROSS JOIN store st
    GROUP BY d.DateKey, p.productKey, st.StoreKey, inv.numberItems, i.basePrice, d.Date
''')
inventory_data = cur.fetchall()
# Insert data into the InventoryFact table
for row in inventory_data:
    date_key = row[0]
    product_key = row[1]
    store_key = row[2]
    num_available = row[3]
    cost_to_store_item_level = row[4]
    cost_to_store_item_level = float(cost_to_store_item_level.replace("$", ""))
    num_cases_purchased_to_date = row[5]
    cost_to_store_case_level = cost_to_store_item_level * 12 * row[6]
    
    cur.execute('''
        INSERT INTO InventoryFact (
            DateKey, ProductKey, StoreKey, NumAvailable, 
            CostToStoreItemLevel, CostToStoreCaseLevel, NumCasesPurchasedToDate
        )
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (date_key, product_key, store_key, num_available, cost_to_store_item_level, cost_to_store_case_level, num_cases_purchased_to_date))


# Ensure data insertion worked
cur.execute('SELECT * FROM InventoryFact LIMIT 20')
rows = cur.fetchall()
for row in rows:
    print(row)

# Commit changes and close the connection
con.commit()
cur.close()
con.close()

Adds the Conformed Products Table

In [ ]:
import sqlite3 as lite
import csv

# Connect to the database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

cur.execute("ALTER TABLE products ADD product_family TEXT")
with open(r'ConformedProducts2.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    for i,row in enumerate(csv_reader):
        if(i == 0):
            continue
        cur.execute("UPDATE products SET productKey = ?, sku = ?, product_Name = ?, product_Class_ID = ?, subcategory = ?, category = ?, department = ?, product_family = ?, size = ?, brand_name = ?, supplier = ? WHERE sku = ?",
            (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[1]))
    con.commit()

# Ensure data insertion worked
cur.execute('SELECT * FROM products LIMIT 20')
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
con.close()

Simulates Another 6 Months of Transactions

In [ ]:
import sqlite3 as lite
import csv
import random
from sqlite3.dbapi2 import Date
import datetime

def readItems(cur, skus, log):
    # No table dropping/creation here, just reading SKUs
    cur.execute("SELECT sku FROM items")
    rows = cur.fetchall()
    for row in rows:
        skus.append(row[0])

def getTypeSKU(cur, type):
    cur.execute("SELECT sku, basePrice FROM items WHERE type = ?", (type,))
    rows = cur.fetchall()
    return random.choice(rows)

def buyItem(cur, count, date, row):
    priceMulti = 1.2
    count += 1
    cur.execute('INSERT INTO sales (customerNum, sku, saleDate, price) VALUES (?, ?, ?, ?)', 
                (count, row[0], date.date().strftime('%Y-%m-%d'), round(float(row[1].strip("$")) * priceMulti, 2)))

def buyRandomItem(cur, count, date, skus):
    sku = random.choice(skus)
    cur.execute("SELECT sku, basePrice FROM items WHERE sku = ?", (sku,))
    rows = cur.fetchall()
    buyItem(cur, count, date, rows[0])

def randomCustomer(dailyCount, customerNumber, date, skus):
    maxItems = 80
    itemCount = 0
    milkChance = 70
    babyFoodChance = 20
    breadChance = 50
    pbChancce = 10
    cerealChance = 5
    diaperChance = 1
    jellyChance = 5

    customerTotal = random.randrange(1, maxItems)
    if random.randrange(1, 100) < milkChance:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Milk'))
        itemCount += 1
        cerealChance = 50
    if itemCount >= customerTotal: return customerTotal
    if random.randrange(1, 100) < cerealChance:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Cereal'))
        itemCount += 1
    if itemCount >= customerTotal: return customerTotal
    if random.randrange(1, 100) < babyFoodChance:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Baby Food'))
        itemCount += 1
        diaperChance = 80
    if itemCount >= customerTotal: return customerTotal
    if random.randrange(1, 100) < diaperChance:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Diapers'))
        itemCount += 1
    if itemCount >= customerTotal: return customerTotal
    if random.randrange(1, 100) < breadChance:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Bread'))
        itemCount += 1
    if itemCount >= customerTotal: return customerTotal
    if random.randrange(1, 100) < pbChancce:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Peanut Butter'))
        itemCount += 1
        jellyChance = 90
    if itemCount >= customerTotal: return customerTotal
    if random.randrange(1, 100) < jellyChance:
        buyItem(cur, customerNumber, date, getTypeSKU(cur, 'Jelly/Jam'))
        itemCount += 1
    if itemCount >= customerTotal: return customerTotal
    for i in range(itemCount, customerTotal):
        buyRandomItem(cur, customerNumber, date, skus)
    return customerTotal

def randomCount(WKDmin, WKDmax, WkEnd, start_date):
    customerNum = random.randint(WKDmin, WKDmax)
    if start_date.weekday() > 4:
        customerNum += WkEnd
    return customerNum

# Connect to database
con = lite.connect(r'jacobs_store.db')
cur = con.cursor()

# Initialize SKU list
skus = []
readItems(cur, skus, False)

# Define date range for new transactions
current_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 6, 30)
delta = datetime.timedelta(days=1)

# Transaction parameters
WKDmin = 1000
WKDmax = 1040
WkEndCount = 75
dailyCount = 0
customerTotal = 0

while current_date <= end_date:
    dailyCount = 0
    customerCount = randomCount(WKDmin, WKDmax, WkEndCount, current_date)
    for c in range(customerCount):
        dailyCount += randomCustomer(dailyCount, c, current_date, skus)
    con.commit()
    current_date += delta
    customerTotal += customerCount

# Ensure data insertion worked
cur.execute('SELECT * FROM sales ORDER BY saleDate DESC LIMIT 20')
rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
con.close()

Calculates the Top 25 Products of Each Store

In [ ]:
import sqlite3 as lite
import pandas as pd

def getSale(path, year):
    # Connect to the database
    con = lite.connect(path)
    cur = con.cursor()

    query = '''
        SELECT 
            sales.sku, 
            products.product_Name AS productName, 
            products.brand_name AS brand_name,
            products.category AS category,
            products.subcategory AS subcategory,
            COUNT(sales.sku) AS NUM 
        FROM sales 
        JOIN products ON sales.sku = products.sku
        WHERE sales.saleDate BETWEEN ? AND ?
        GROUP BY sales.sku
        ORDER BY NUM DESC
    '''
    #Q1 1/1 - 3/31
    start_date = f"{year}-01-01"
    end_date = f"{year}-03-31" 

    cur.execute(query, (start_date, end_date))
    sales = cur.fetchall()

    # Clean up
    cur.close()
    con.close()

    return sales

def addSales(db): 
    salesDB_2023 = getSale(db, 2023)
    salesDB_2024 = getSale(db, 2024) 

    # Convert sales data to DataFrames
    # 2023
    db_2023 = pd.DataFrame(salesDB_2023, columns=['sku', 'productName', 'brand_name', 'category', 'subcategory', 'NUM'])
    # 2024
    db_2024 = pd.DataFrame(salesDB_2024, columns=['sku', 'productName', 'brand_name', 'category', 'subcategory', 'NUM'])
    
    # Combine the data for aggregated totals
    combined = pd.concat([db_2023, db_2024]) 

    aggregated = combined.groupby(['sku', 'productName'], as_index=False).agg({'NUM': 'sum'})
    
    aggregated = aggregated.sort_values(by='NUM', ascending=False)

    return aggregated, db_2023, db_2024


def printReport(db):
    totSales, db_2023, db_2024 = addSales(db)
    
    # Get the top 25 combined sales
    combined = totSales.head(25)
    
    # Rank the sales within each year (2023 and 2024)
    db_2023['rank_2023'] = db_2023['NUM'].rank(method='first', ascending=False)
    db_2024['rank_2024'] = db_2024['NUM'].rank(method='first', ascending=False)

    # Rename NUM columns for clarity before merging
    db_2023 = db_2023.rename(columns={'NUM': 'NUM_2023'})
    db_2024 = db_2024.rename(columns={'NUM': 'NUM_2024'})
    
    # Merge with combined to align the order
    dbOrder_2023 = combined.merge(db_2023[['sku', 'productName', 'brand_name', 'category', 'subcategory', 'NUM_2023', 'rank_2023']], 
                                  on=['sku', 'productName'], how='left').fillna(0)
    dbOrder_2024 = combined.merge(db_2024[['sku', 'productName', 'brand_name', 'category', 'subcategory', 'NUM_2024', 'rank_2024']], 
                                  on=['sku', 'productName'], how='left').fillna(0)
    
    # Reorder db_2023 and db_2024 according to the order in combined
    dbOrder_2023 = dbOrder_2023.set_index(['sku', 'productName']).reindex(combined.set_index(['sku', 'productName']).index).reset_index()
    dbOrder_2024 = dbOrder_2024.set_index(['sku', 'productName']).reindex(combined.set_index(['sku', 'productName']).index).reset_index()

    # Print the results for 2023
    print("Combined Top 25, Year 2023")
    print("sku | productName | brandName | category | subcategory | rank | #sold")
    for _, sale in dbOrder_2023.iterrows():
        print(f"{sale['sku']} | {sale['productName']} | {sale['brand_name']} | {sale['category']} | {sale['subcategory']} | {int(sale['rank_2023'])} | {int(sale['NUM_2023'])}")
    
    # Print the results for 2024
    print("Combined Top 25, Year 2024")
    print("sku | productName | brandName |category | subcategory | rank | #sold")
    for _, sale in dbOrder_2024.iterrows():
        print(f"{sale['sku']} | {sale['productName']} | {sale['brand_name']}| {sale['category']} | {sale['subcategory']} | {int(sale['rank_2024'])} | {int(sale['NUM_2024'])}")

print('Tim')
db = 'tims_store.db'
printReport(db)

print('\nJacob')
db = 'jacobs_store.db'
printReport(db)

Calculates the Top 25 Product Categories of Each Store

In [ ]:
import sqlite3 as lite
import pandas as pd

def getSale(path, year):
    # Connect to the database
    con = lite.connect(path)
    cur = con.cursor()

    query = '''
        SELECT 
            products.category AS category,
            products.brand_name as brand_name
            COUNT(sales.sku) AS NUM
        FROM sales 
        JOIN products ON sales.sku = products.sku
        WHERE sales.saleDate BETWEEN ? AND ?
        GROUP BY products.category
        ORDER BY NUM DESC
    '''
    # Q1 1/1 - 3/31
    start_date = f"{year}-01-01"
    end_date = f"{year}-03-31" 

    cur.execute(query, (start_date, end_date))
    sales = cur.fetchall()

    # Clean up
    cur.close()
    con.close()

    return sales

def addSales(db): 
    salesDB_2023 = getSale(db, 2023)
    salesDB_2024 = getSale(db, 2024) 

    # Convert sales data to DataFrames
    # 2023
    db_2023 = pd.DataFrame(salesDB_2023, columns=['category', 'NUM'])
    # 2024
    db_2024 = pd.DataFrame(salesDB_2024, columns=['category', 'NUM'])
    
    # Combine the data for aggregated totals
    combined = pd.concat([db_2023, db_2024]) 

    # Aggregated totals by category
    aggregated = combined.groupby('category', as_index=False).agg({'NUM': 'sum'})
    
    aggregated = aggregated.sort_values(by='NUM', ascending=False)

    return aggregated, db_2023, db_2024


def printReport(db):
    totSales, db_2023, db_2024 = addSales(db)
    
    # Get the top 25 combined sales by category
    combined = totSales.head(25)
    
    # Rank the sales within each year (2023 and 2024)
    db_2023['rank_2023'] = db_2023['NUM'].rank(method='first', ascending=False)
    db_2024['rank_2024'] = db_2024['NUM'].rank(method='first', ascending=False)

    # Rename NUM columns for clarity before merging
    db_2023 = db_2023.rename(columns={'NUM': 'NUM_2023'})
    db_2024 = db_2024.rename(columns={'NUM': 'NUM_2024'})
    
    # Merge with combined to align the order
    dbOrder_2023 = combined.merge(db_2023[['category', 'NUM_2023', 'rank_2023']], 
                                  on='category', how='left').fillna(0)
    dbOrder_2024 = combined.merge(db_2024[['category', 'NUM_2024', 'rank_2024']], 
                                  on='category', how='left').fillna(0)
    
    # Reorder db_2023 and db_2024 according to the order in combined
    dbOrder_2023 = dbOrder_2023.set_index('category').reindex(combined.set_index('category').index).reset_index()
    dbOrder_2024 = dbOrder_2024.set_index('category').reindex(combined.set_index('category').index).reset_index()

    # Print the results for 2023
    print("Combined Top 25, Year 2023")
    print("category | rank | #sold")
    for _, sale in dbOrder_2023.iterrows():
        print(f"{sale['category']} | {int(sale['rank_2023'])} | {int(sale['NUM_2023'])}")
    
    # Print the results for 2024
    print("Combined Top 25, Year 2024")
    print("category | rank | #sold")
    for _, sale in dbOrder_2024.iterrows():
        print(f"{sale['category']} | {int(sale['rank_2024'])} | {int(sale['NUM_2024'])}")

print('Tim')
db = 'tims_store.db'
printReport(db)

print('\nJacob')
db = 'jacobs_store.db'
printReport(db)